* review Lab 10a

# Six coupled differential equations and critical comparison of different solution schemes
In this notebook we will setup a solution for nuclear burning of H to He via the CN cycle. This is a real-life problem that allows us to explore different solution techniques and their advantages and disadvantages. It will become clear why there is not one approach to fit all problems.

As with the skydiver problem I will briefly motivate the physics behind the problem, and soon arrive at the set of equations to be studied.

## Nuclear network 

A nuclear network code is another example of solving a coupled system of ODEs. Imagine a situation where you have a number of constituents which can react with each other pairwise and form a new constituent. That's a network of constituents in which the transmutation through reactions is described by rate equations. 

Energy in stars is generated by nuclear reactions between different isotopes of elements. This is described by nuclear network code.
The first  reactions of the CN cycle for hydrogen burning are:

array index | reaction
------------|---------
0 |$^{12}\mathrm{C}+\mathrm{p} \rightarrow ^{13}\mathrm{N}+\mathrm{\gamma}$ followed immediately by the $\beta$ decay of $^{13}\mathrm{N}$ to $^{13}\mathrm{C}$
1 | $^{13}\mathrm{C}+\mathrm{p} \rightarrow ^{14}\mathrm{N}+\mathrm{\gamma}$ and
2 |$^{14}\mathrm{N}+\mathrm{p} \rightarrow ^{15}\mathrm{O}+\mathrm{\gamma}$ followed immediately by the $\beta$ decay of $^{15}\mathrm{O}$ to $^{15}\mathrm{N}$
3 | $^{15}\mathrm{N}+ \mathrm{p} \rightarrow ^{12}\mathrm{C}+\mathrm{\alpha}$ which closes the CN cycle.

These are the reactions of the important CN cycle that is the _catalytic_ reaction chain to transform H into He in massive stars.

![ChartCNO](./images/ChartCNO.png)

The rate coefficients $<\sigma v>$ describe how quickly the transmutations occur. They depend on the temperature. To described this generally you need to take the coeffiecient as a function of T from a table and interpolate appropriately. Here you are asked to initially only enter the rate coefficients for one constant T.

Create a nuclear network code for the CN cycle that operates at a fixed temperature of $T=9\times10^{7}\mathrm{K}$ and a density of $\rho = 100 \mathrm{g/cm^3}$. 

The evolution of each **species** or **constituent** is governed by a rate equation that has on the right-hand side the sum of all production and destruction terms. In terms of the number density $N_j$ of species $j$ we collect all
production and destruction terms of reactions of the type $k + l
\rightarrow j + n$ 
$$
\frac{dN_j}{dt} = N_k N_l<\sigma v>_{kl,j} - N_j N_m <\sigma v>_{jn,o} 
$$
where $<\sigma v>$ is the reaction rate (the product of the cross section and the
relative velocity in the center-of-mass system averaged over the
appropriate distribution function) that can be obtained from the [NACRE compilation](http://www.astro.ulb.ac.be/nacreii). 

The number density is expressed in terms of a number fraction or mole
fraction by $N=Y \rho  N_\mathrm{A}$ where $N_\mathrm{A}$ is
the Avogadro number. The rate given in the NACRE tables is in terms of 
$  N_\mathrm{A} <\sigma v>$.

Therefore we have six equations, one for each species. On the left is the rate of change, and on the right is the sum of the production and destruction terms. These are, in terms of mole fractions

$$
\begin{eqnarray}
\frac{Y(p)}{dt}   &=&  -r_0 Y(C12)Y(p) &- r_1 Y(C13)Y(p) &- r_2 Y(N14)Y(p) &-r_3 Y(N15)Y(p) \\
\frac{Y(C12)}{dt} &=&  -r_0 Y(C12)Y(p) &&&+ r_3 Y(N15)Y(p) \\
\frac{Y(C13)}{dt} &=&  +r_0 Y(C12)Y(p) &- r_1 Y(C13)Y(p)&& \\
\frac{Y(N14)}{dt} &=&  &+r_1 Y(C13)Y(p)&- r_2 Y(N14)Y(p)& \\
\frac{Y(N15)}{dt} &=&  &&+r_2 Y(N14)Y(p) &- r_3 Y(N15)Y(p) \\
\frac{Y(He4)}{dt} &=&   && &+ r_3 Y(N15)Y(p) 
\end{eqnarray}
$$

The rates in this equation set are $r_i = \rho N_\mathrm{A} <\sigma v>_i$  which takes care of going from number densities $N$ to mole fractions $Y$. The inital abundances in terms of the mass fraction $X$ according to the solar abundance distribution are provided in the file `iniab1.4E-02As09.ppn`. Molar fractions $Y$ are obtained from mass fractions by $Y=X/A$ ,with $A$ the atomic mass number. 

Let's solve this network with the goal to determine the time evolution of each species.

### Model solution

#### Read initial abundance file
* This set of ODEs is an initial-value problem, so I need initial values for all species. These are compiled for you in the file `data/iniab1.4E-02As09.ppn`.
* The file has four columns which contain the charge number, the element name, the mass number and the mass fraction.

In [ ]:
%pylab ipympl

In [ ]:
%%bash
cat data/iniab1.4E-02As09.ppn

### There are two options to do this
#### Option 1

In [ ]:
# read initial abundance file
#f.close()
f_ini=open('data/iniab1.4E-02As09.ppn')

In [ ]:
ind=[];elem=[];A=[];X=[]   # we will sort the row items into these three lists
for i,line in enumerate(f_ini.readlines()):
    a,b,c,d=line.split()
    # the first column in the file contains the charge 
    ind.append(i)  # number; we don't need it, but an index variable 
    elem.append(b) # would be useful
    A.append(c)
    X.append(d)

In [ ]:
# close the file handle!
f_ini.close()

In [ ]:
# read ini abund tester
for i in range(len(ind)):
    print (ind[i],elem[i],A[i],X[i])

In [ ]:
#loadtxt?

#### Option 2
In Lab 11a it suggested to use `numpy.loadtxt` introduced in notebook 3. We can use that here as well. Have a look at the docstring first!

In [ ]:
Z,A,X = loadtxt('data/iniab1.4E-02As09.ppn',unpack=True,usecols=(0,2,3))           # select columns with floats and read those
elem = loadtxt('data/iniab1.4E-02As09.ppn',unpack=True,usecols=(1),dtype='str')    # select column with character and read that separately. 

In [ ]:
# tester
for a,b,c,d in zip(Z,elem,A,X):
    print(a,b,c,d)

Calculate intial conditions in right units (molar fractions):

In [ ]:
# 
X0=array(X,float)
A=array(A,float)
Y0 = X0/A

In [ ]:
# initial molar density for all species:
print(Y0)

#### RHS of network

The rates $<\sigma v>$ will be held in an array `rate` with length 4. Initially we will copy-paste the values from the online tables for the given temperature.

In [ ]:
global rate

rate=[7.36E-06]       # C12(p,g)
rate.append(3.52E-05) # C13(p,g)
rate.append(2.36E-07) # N14(p,g)
rate.append(2.03E-02) # N15(p,a)

rate = array(rate)
print(rate)
# we can see from the large difference in value for the
# rates that this set of ODEs is stiff; it may
# require short time steps, or an implicit solution scheme


In [ ]:
# this is the indexing of the initial abundance vector:
for i in ind:
    print(i, elem[i], A[i])

In [ ]:
def react_terms(y):
    terms=[]
    terms.append(rate[0]*y[2]*y[0]) # 0 C12(p,g)
    terms.append(rate[1]*y[3]*y[0]) # 1 C13(p,g)
    terms.append(rate[2]*y[4]*y[0]) # 2 N14(p,g)
    terms.append(rate[3]*y[5]*y[0]) # 3 N15(p,a)
    return array(terms)

In [ ]:
def f_rhs(y,t):
    '''Provide RHS for CN network equations''' 

    terms = react_terms(y)

    dh1_dt  =  -terms.sum()
    dhe4_dt =   terms[3]
    dc12_dt =  -terms[0] + terms[3]
    dc13_dt =  -terms[1] + terms[0]
    dn14_dt =  -terms[2] + terms[1]
    dn15_dt =  -terms[3] + terms[2]
    
    return array([dh1_dt,dhe4_dt,dc12_dt,dc13_dt,dn14_dt,dn15_dt])

In [ ]:
rho = 100. #cgs
1./(1.e-2*rate[1]*rho) #in seconds

In [ ]:
# tester react_terms: term 0 - C12(p,g)
terms = react_terms(Y0)
terms[0], rate[0], Y0[0], Y0[2]

In [ ]:
# tester f_rhs
t = 0
f_rhs(Y0,t)

#### Plotting utilities

In [ ]:
# from the data in the initial abundance file we can create
# element name strings ("everything can only be in one place"):
[elem[i]+str(int(A[i])) for i in range(len(A))]

In [ ]:
# in order to plot the lines of all elements setup a linestyle list
linestyles = ['-', '--', '-.', ':']
# we only have 4 linestyles, with the modulus function we can get
# a linestyle for any i
i = randint(100); print(i)
print(linestyles[mod(i,4)])

#### Explicit solution

In [ ]:
Ynp1 = lambda y: y + dt * f_rhs(y,t)

In [ ]:
# intial conditions in right units:
X0=array(X,float)
A=array(A,float)
Y0 = X0/A

In [ ]:
dt        = 35.               # time step
te        = arange(0,2.e3,dt) # time array for explicit soln 
markevery = 3                # parameter to plot a glyph only every 
                              # markervery data points

In [ ]:
Yn = []
yy = Y0
Yn.append(yy)
for i in range(len(te)-1):
    yy = Ynp1(yy)
    Yn.append(yy)   
Yn = array(Yn)

In [ ]:
close(1);figure(1)
for i in range(len(A)):
    plot(te,log10(Yn.T[i]*A[i]),label=elem[i]+str(int(A[i])),\
         linestyle=linestyles[mod(i,4)],marker=i+5,markevery=markevery)
legend(loc=4)
ylabel('mass fraction');xlabel('time/s')
#ylim(-2e-6,2e-6)

The problem with the explicit solution is that it requires very small time steps to resolve the fast destruction of $^{15}N$. Let's see how the odeint method deals with this.

#### Odeint solution

In [ ]:
from scipy import integrate

In [ ]:
t = te   # to use same time array as in explicit soln only use this cell
         # and use dt and markevery

In [ ]:
dt        = 30.
t         = arange(0,2.e3,dt)  
markevery = 1

In [ ]:
Y=integrate.odeint(f_rhs,Y0,t)

In [ ]:
close(2);figure(2)
for i in range(len(A)):
    plot(t,log10(Y.T[i]*A[i]),label=elem[i]+str(int(A[i])),\
         linestyle=linestyles[mod(i,4)],marker=i+5,markevery=markevery)
legend(loc=4)
ylabel('mass fraction');xlabel('time/s')
#ylim(-2e-6,2e-6)

The _odeint_ package uses quite obiously some more advanced solution methods. How are such more advanced methods constructed? There are various strategies. Runge-Kutta is an example of a higher-order scheme that evaluates approximately the RHS at yet to be determined locations of the next time step. In the coupled network equation encountered in this example the problem is not just overall accuracy, but the fact that the rate coefficient are different by orders of magnitude. The p capture of $^{15}N$ is 5 orders of magnitude faster than the p capture of $^{14}N$, and the result is a fast decline initially of $^{15}N$ that dictates the time step for the entire calculation. Such coupled ODEs with RHS terms with large differences in their coedfficients are called _stiff_. There is one golden bullet to deal with _stiff_ problems, which is the implicit solution described next.

## Implicit ODE with Newton-Raphson
The Newton-Raphson method has been introduced before when used with Sympy. It can be used as well for an alternative
approach to integrating ODEs, such as the sky diver
problem. So far we have used an _explicit_ forward integration which
involves evaluating the RHS of the ODE using the know velocity
$ v_n $ at the present time to get the value of the
velocity at the next time step $ n+1 $. In cases where
we are interested in the longer-term behaviour, and where we are
required to take larger time steps this may not work very well. In
that case we want to evaluate the RHS for $ v(n+1) $, and we have
an _implicit_ solution scheme. Especially for large coupled systems of
ODEs implicit schemes often provide the numerical stability to find a
solution at all. The implicit solution involves an iterative approach,
and we can use again the Newton-Raphson method. The difference of an implicit method is to evalute the RHS not at the know time, e.g. $y_\mathrm{n}$ but instead at the not yet known time at step $\mathrm{n+1}$. Thus, instead of solving

$$\frac{y_\mathrm{n+1} - y_\mathrm{n}}{h} = f(t,y_\mathrm{n})$$

we solve

$$\frac{y_\mathrm{n+1} - y_\mathrm{n}}{h} = f(t, y_\mathrm{ n+1})$$

$h$ is again the time step $dt$ in most cases, or whatever the derivative quantity is.

We now cannot solve the discretized equation for $y_\mathrm{n+1}$ analytically anymore and therefore have to solve numerically. We can reshape this as a root-finding problem. We arrange all terms on one side and define this as a new function: 

$$G(y_\mathrm{n+1}) = y_\mathrm{n+1} - y_\mathrm{n} - h  f(t,y_\mathrm{ n+1})$$

The task is to find the root of $G(y_\mathrm{n+1})$, i.e. the solution of $G(y_\mathrm{n+1}) = 0$, which can be done using Newton-Raphson.

As you can see the basic strategy is to put everything on one side,
give it a name $ G(y_{n+1})$, set this function to zero and
solve it by Newton-Raphson. The beauty is that this idea can be easily
expanded to systems of ODEs with many variables. In physics this 
method is for example used for large reaction rate networks involving
thousands of isotopes, solving radiation transfer problems or computing the evolution of stars in spherical symmetry according to the conservation laws of mass, momentum and energy.

At the end of this notebook the extension to multiple dimensions is described for those interested. But the implementation is a bit more advanced. Here will just implement the implicit solution for the equation for $^{15}N$, and explore the resulting behaviour for the first $2000\mathrm{s}$ when the evolution of $^{15}N$ is dominated by the destruction due to p capture, and the production from the $^{14}N(p,\gamma)$ rate is unimportant.

#### Brief review of Newton-Raphson method
You have seen this method already in course notebook 7. Let's recast it here for application to solve ODEs. In order to not get confused with the indices, let's for the moment replace $z = y_\mathrm{n+1}$. In this way it is more clear that the root finding iteration does not go over the index $n$. For the purpise of root finding for _one_ timestep $n$ does not change. 

Then the equation for which we need to find the root is
$$G(z) = z - y_\mathrm{n} - h  f(t,z)$$
where $y_\mathrm{n}$ is a constant, and we need to find $z$ so that $G(z)=0$. We start with an initial guess $z'$ which is usually taken to be the last known solution  $z' = y_\mathrm{n}$. Then, with $\delta = z - z'$ the Taylor expansion to first order gives
$$
G(z) = G( z' + \delta) =  G(z') +\frac{dG(z')}{dz'}\delta 
$$
which as to be set to 0 and therefore 
$$
\delta = - \frac{G(z')}{\frac{dG(z')}{dz'}}
$$
Calculating this once gives an initial estimate for $\delta$, and $z = z' + \delta$ is an initial estimate for $z$. We then set $z' = z$ as a better first guess and repeat the updating procedure, e.g. calculate a new $\delta$. This is done until the termination condition $\delta < \delta_\mathrm{max}$ is satisfied,  where $\delta_\mathrm{max}$ a user-set parameter to control precision. Once this procedure has converged we have found $y_\mathrm{n+1} = z$ and proceed to the next time step.

In [ ]:
# in order to calculate the derivative we will need
def derivs(f,x,h=1):
    '''
    Return 1st order numerical derivative
    
    f : function
    x : array like x value to evaluate derivative
    h : interval in x for derivative
    '''    
    dfdx = (f(x+h) -f(x))/h 
    return dfdx

In [ ]:
# intial abundance only for N15, index 5 is N15
yn = Y0[5]; yn

In [ ]:
# the RHS function f is already available: f_rhs(yn,t)
# we want to reuse it(!), but here it should only return the RHS
# for equation 6 for N15; this is accomplished with a function wrapper
def f_rhs_oneeq(z,Y0,ind_oneeq=5):
#    Y = Y0   ### WARNING ### the = operator is a pointer that does not make a copy
    Y = copy(Y0)  
    Y[ind_oneeq] = z
    return f_rhs(Y,t)[ind_oneeq]

In [ ]:
f_rhs_oneeq(0.1*yn,Y0),Y0[5]

In [ ]:
# using the nomenclature from above we use
# yn = y_n and dt = h
G = lambda z: z - yn - dt * f_rhs_oneeq(z,Y0)

In [ ]:
# zp = z', the initial guess is
zp = yn

What dt should we take? We saw that the explicit solution was working for `dt = 10` but not for `100`. Let's see what the implicit solution does for `dt = 100`.

In [ ]:
dt = 100

Now, let's first check how G looks like, what are we trying to find the root of?

In [ ]:
x=linspace(zp-2*zp,zp+2*zp,3)
# G(z) as defined does not operate on arrays -> use list comprehension
[G(z) for z in x]

In [ ]:
close(4);figure(4)
plot(x,[G(z) for z in x],':')
axhline(color='k',linestyle='dashed')
vlines(zp,G(x[0]),G(x[-1]),color='orange',linestyle='dotted')
plot(zp,G(zp),'h')
title('$G(z)$')
xlabel('$z$')
ylabel('$G$')

The orange line marks the position of `zp` - the initial guess. The root of $G(z)$ is at `z < zp`. Let's calculate $\delta$ and apply it. 

In [ ]:
z = yn; z

In [ ]:
# one NR iteration, repeat until satisfactory solution is found
# x is the estimate for v_n+1 and accepted as solution if 
# G(xx) < eps1 and delta < eps2, where eps1 and eps2 are 
# suitably small limits
zp = z; h = zp/10.            # set initial guess and h for derivs
delta = -G(zp)/derivs(G,zp,h)
z = zp + delta
print(z,G(z),delta)
plot(z,G(z),'h')

Note that because in this case G is linear and because we have only one equation we find the root in one step to machine precision.

In [ ]:
# initialize again and put in loop
# tmax = 200.;dt = 10    # to use same t array as in explicit soln comment this line
tmax = te[-1]
yn = Y0[5]
z = yn
YN15 = []; ti = []; tt = 0   # ti is the time array for implicit method
YN15.append(z); ti.append(tt)
print(4*"%10s  "%('time','z','G(z)','delta'))  # print table header
print(4*"%10.3e  "%(tt,z,0.,0.))               # print initial condition   
while tt < tmax:
    delta = 1
    while delta > 0.001:
        zp = z; h = zp/10.            # set initial guess and h for derivs
        delta = -G(zp)/derivs(G,zp,h)
        z = zp + delta
        #print(4*"%10.3e  "%(tt,z,G(z),delta))  # print and check for small step numbers
    YN15.append(z)
    tt += dt
    ti.append(tt)
    yn = z         # this is a constant in G, but needs to be update after 
                   # each converged time step
YN15 = array(YN15)

In [ ]:
close(7);figure(7)
indN15 = 5; markevery = markevery
plot(te,log10(Yn.T[indN15]*A[indN15]),label="explicit",\
         linestyle=linestyles[mod(indN15,4)],marker=7,markevery= markevery)
plot(t,log10(Y.T[indN15]*A[indN15]),label="odeint",\
         linestyle=linestyles[mod(indN15,4)],marker=7,markevery= markevery)
plot(ti,log10(YN15*A[indN15]),label="implicit",\
         linestyle=linestyles[2],marker=9,markevery= markevery)
legend(loc=1)
ylabel('mass fraction');xlabel('time/s')
#ylim(-2e-6,2e-6)

### Observations

* For intermediate dt, e.g. 35, the explicit solution assumes the slope at the beginning of each dt interval, which is in this case always too large and as a result the $^{15}N$ abundance drops too fast. For the implicit solution the slope at the end of the interval is adopted for each interval, which is always too small. As a result the implicit solution drops off too slowly. The odeint library implements a higher-order solution which lies inbetween.
* For small dt, e.g. 10, all three solution schemes agree well. Making the time step small increases the accuracy of any scheme, usually.
* For very small time step the odeint methods start to show some artefacts that may have to do with precision. The other two schemes do not.
* For large dt, e.g. 100, the explicit scheme produces negative values and goes off the rail. However, the implicit solution still gives reasonable results, even if less accurate. This is the special property of the implicit discretization. It is very robust, even for sets of 1000s of equations. It is only first order accurate, but it is fast and can be done with small time steps.

## Implicit solution for systems of ODEs
**Advanced topic, not for assignments or final exam**

Below we write out the implicit solution scheme for multiple coupled ODEs, for the example of the skydiver when we integrate the height $h$ and the velocity $v$ simultaneously. In the case of the skydiver the implicit solution is not a particular benefit as that system is not stiff. However, we can show how analytic derivatives can be used in the Jacoby matrix.

$$ v_{n+1} = v_{n} +dt( -g +  k v_{n+1}^2)\\
h_{n+1} = h_{n} +dt v_{n+1}$$

and in terms of the function we have to find the root for

$$ G_1(v_{n+1}) = - v_{n+1} + v_{n} +dt( -g +  k v_{n+1}^2)\\
G_2(h_{n+1}) = -h_{n+1} + h_{n} +dt v_{n+1}$$
 
and with $Y_1 = v_{n+1}$ and $Y_2 = h_{n+1}$ and $c_1 = v_{n} - dt\, g$ and $c_2 = h_{n}$

$$ G_1(Y_1) = - Y_1 + c_1 + dt\,k\, Y_1^2\\
G_2(Y_2) = -Y_2 + c_2 + dt\, Y_1 $$

or 

$$ \vec{G}(\vec{Y}) = \matrix{F} \cdot \vec{Y} + \vec{c}\\
\matrix{F} = \pmatrix{ (-1 + dt\,k) & 0  \\ 
                       dt & -1  }
$$ 

Now the Newton-Raphson for finding the root of $\vec{G}(\vec{Y})=0$ would look like this

$$
\vec{G}'(\vec{Y}) \, \vec{\delta} = -\vec{G}(\vec{Y}) 
$$

where $\vec{Y}$ is to start with the estimate of the solution for the next time step, and $\vec{G}'(\vec{Y})$ is the Jacoby matrix. It has on our case the following elements:

$$
\vec{G}'(\vec{Y}) = \pmatrix{ \frac{\partial G_1}{\partial Y_1} & \frac{\partial G_1}{\partial Y_2}  \\ 
                     \frac{\partial G_2}{\partial Y_1} & \frac{\partial G_2}{\partial Y_2}  } $$

and so, for each time step we have to do a number of Newton-Raphson iterations, each of which involves solving the linear algebra problem

$$
\vec{G}'(\vec{Y}) \cdot \vec{\delta} = -\vec{G}(\vec{Y}) 
$$

which we can solve, i.e. find $\vec{\delta}$, using Gaussian elimination.

More generally, for a system with $N$ ODEs we would have $N$ variables:

$$\vec{Y} =\left(\begin{array}{c}
Y_1 \\
Y_2 \\
\vdots \\
Y_i \\
\vdots \\
Y_N \\
\end{array}\right)
$$

The Jacoby matrix is 
$$
\vec{G}'(\vec{Y}) =\frac{\partial \vec{G}(\vec{Y})}{\partial \vec{Y}}
=\left(\begin{array}{c}
\vec{G}_1'(\vec{Y}) \\
\vec{G}_2'(\vec{Y}) \\
\vdots \\
\vec{G}_i'(\vec{Y}) \\
\vdots \\
\vec{G}_N'(\vec{Y})
\end{array}\right) ,
$$
and contains the row elements
$$
  \frac{\partial G_\mathrm{i}(\vec{Y}) }  {\partial \vec{Y}}=
  \vec{G}_i'(\vec{Y})=\left( \frac{\partial G_i}{\partial Y_{1}},
    \frac{\partial G_i}{\partial Y_{2}},\dots,
      \frac{\partial G_i}{\partial Y_{i}},\dots,
      \frac{\partial G_i}{\partial Y_{N}}\right) .
$$
which defines for each iteration in each time step a linear algebra problem with $N$ variables.

In many real problems the Jacoby matrix may be _sparse_ which means it is only partially populated and lots of elements are zero. In that case special numerical solution techniques are used, e.g. _sparse solvers_